<a href="https://colab.research.google.com/github/Donggeon2960/AutoDrive-ML/blob/main/yolov8_youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics yt-dlp opencv-python

import cv2
import os
from ultralytics import YOLO
from IPython.display import Video, display

def download_and_analyze_youtube_video(youtube_url):
    """
    YouTube 영상을 직접 다운로드하고 YOLO 분석
    """
    print("⬇️ YouTube 영상 다운로드 중...")

    # YouTube 영상 다운로드 (yt-dlp 직접 명령어 사용)
    os.system(f'yt-dlp -f "bestvideo+bestaudio/best[height<=720]" --merge-output-format mp4 "{youtube_url}"')

    # 다운로드된 mp4 파일 찾기
    video_file = None
    for file in os.listdir("/content"):
        if file.endswith(".mp4"):
            video_file = file
            print(f"📁 다운로드된 영상: {file}")
            break

    if not video_file:
        print("❌ 다운로드된 영상을 찾을 수 없습니다.")
        return None

    # YOLO 모델 로드
    print("🤖 YOLO 모델 로드 중...")
    model = YOLO("yolov8n.pt")

    # 비디오 캡처 설정
    print("🎬 영상 분석 시작...")
    video_path = f"/content/{video_file}"
    cap = cv2.VideoCapture(video_path)

    # 비디오 정보 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps

    print(f"📹 영상 정보:")
    print(f"   해상도: {width}x{height}")
    print(f"   FPS: {fps:.1f}")
    print(f"   총 프레임: {total_frames}")
    print(f"   길이: {duration:.1f}초")

    # 결과 저장 설정
    output_path = "result_video_youtube.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 분석 통계
    frame_count = 0
    detection_count = 0
    frames_with_objects = 0

    # 프레임별로 추론 및 저장
    print("🔍 YOLO 분석 진행 중...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8 추론
        results = model(frame, verbose=False, conf=0.5)

        # 탐지된 객체 수 계산
        if len(results) > 0 and results[0].boxes is not None:
            num_detections = len(results[0].boxes)
            detection_count += num_detections
            if num_detections > 0:
                frames_with_objects += 1

        # 결과 이미지 (bounding box 포함)
        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        frame_count += 1

        # 진행률 표시 (10% 단위)
        if frame_count % (total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"진행률: {progress:.0f}%")

    # 리소스 해제
    cap.release()
    out.release()

    # 분석 결과 통계
    print("\n" + "="*50)
    print("📊 분석 완료! 결과 통계")
    print("="*50)
    print(f"총 처리 프레임: {frame_count}")
    print(f"총 탐지된 객체: {detection_count}")
    print(f"객체가 탐지된 프레임: {frames_with_objects}")
    if frame_count > 0:
        print(f"프레임당 평균 탐지 수: {detection_count/frame_count:.2f}")
        print(f"객체 탐지율: {(frames_with_objects/frame_count)*100:.1f}%")

    print(f"\n✅ 결과 영상 저장: {output_path}")

    # 결과 영상 표시
    print("🎥 결과 영상 재생:")
    return output_path

def analyze_local_video(video_path):
    """
    로컬 비디오 파일 YOLO 분석 (업로드된 파일용)
    """
    if not os.path.exists(video_path):
        print(f"❌ 파일을 찾을 수 없습니다: {video_path}")
        return None

    # YOLO 모델 로드
    print("🤖 YOLO 모델 로드 중...")
    model = YOLO("yolov8n.pt")

    # 비디오 캡처 설정
    print("🎬 영상 분석 시작...")
    cap = cv2.VideoCapture(video_path)

    # 비디오 정보 가져오기
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps if fps > 0 else 0

    print(f"📹 영상 정보:")
    print(f"   해상도: {width}x{height}")
    print(f"   FPS: {fps:.1f}")
    print(f"   총 프레임: {total_frames}")
    print(f"   길이: {duration:.1f}초")

    # 결과 저장 설정
    output_path = "result_video_local.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 분석 통계
    frame_count = 0
    detection_count = 0
    frames_with_objects = 0

    # 프레임별로 추론 및 저장
    print("🔍 YOLO 분석 진행 중...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # YOLOv8 추론
        results = model(frame, verbose=False, conf=0.5)

        # 탐지된 객체 수 계산
        if len(results) > 0 and results[0].boxes is not None:
            num_detections = len(results[0].boxes)
            detection_count += num_detections
            if num_detections > 0:
                frames_with_objects += 1

        # 결과 이미지 (bounding box 포함)
        annotated_frame = results[0].plot()
        out.write(annotated_frame)

        frame_count += 1

        # 진행률 표시 (10% 단위)
        if total_frames > 0 and frame_count % max(1, total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"진행률: {progress:.0f}%")

    # 리소스 해제
    cap.release()
    out.release()

    # 분석 결과 통계
    print("\n" + "="*50)
    print("📊 분석 완료! 결과 통계")
    print("="*50)
    print(f"총 처리 프레임: {frame_count}")
    print(f"총 탐지된 객체: {detection_count}")
    print(f"객체가 탐지된 프레임: {frames_with_objects}")
    if frame_count > 0:
        print(f"프레임당 평균 탐지 수: {detection_count/frame_count:.2f}")
        print(f"객체 탐지율: {(frames_with_objects/frame_count)*100:.1f}%")

    print(f"\n✅ 결과 영상 저장: {output_path}")

    return output_path

# 메인 실행 부분
print("🎬 YouTube YOLO 분석기 (간단 버전)")
print("="*50)

mode = input("\n선택하세요:\n1. YouTube URL 입력\n2. 파일 업로드\n\n선택 (1 또는 2): ")

if mode == "1":
    # YouTube URL 입력
    youtube_url = input("\nYouTube URL을 입력하세요: ")

    result_path = download_and_analyze_youtube_video(youtube_url)

    if result_path and os.path.exists(result_path):
        # 결과 영상 표시
        display(Video(result_path, embed=True, width=800))
    else:
        print("❌ 분석 실패")

elif mode == "2":
    # 파일 업로드
    print("\n📁 비디오 파일을 업로드하세요:")
    try:
        from google.colab import files
        uploaded = files.upload()

        if uploaded:
            video_file = list(uploaded.keys())[0]
            print(f"📹 업로드된 파일: {video_file}")

            result_path = analyze_local_video(video_file)

            if result_path and os.path.exists(result_path):
                # 결과 영상 표시
                display(Video(result_path, embed=True, width=800))
            else:
                print("❌ 분석 실패")
        else:
            print("❌ 파일이 업로드되지 않았습니다.")

    except ImportError:
        # Colab이 아닌 환경
        video_path = input("비디오 파일 경로를 입력하세요: ")
        result_path = analyze_local_video(video_path)

        if result_path and os.path.exists(result_path):
            print(f"✅ 결과 영상: {result_path}")
        else:
            print("❌ 분석 실패")

else:
    print("❌ 잘못된 선택입니다.")

print("\n🎉 작업 완료!")

🎬 YouTube YOLO 분석기 (간단 버전)

선택하세요:
1. YouTube URL 입력
2. 파일 업로드

선택 (1 또는 2): 1

YouTube URL을 입력하세요: https://www.youtube.com/watch?v=54nVQ-8oUzs
⬇️ YouTube 영상 다운로드 중...
📁 다운로드된 영상: 근데 속도 미쳤잖아ㅋ 실제 레이싱에서 나온 '벽 타기!'  결과를 완전히 뒤집어... '세계가 놀랐다' [54nVQ-8oUzs].mp4
🤖 YOLO 모델 로드 중...


100%|██████████| 6.25M/6.25M [00:00<00:00, 109MB/s]


🎬 영상 분석 시작...
📹 영상 정보:
   해상도: 1920x1080
   FPS: 30.0
   총 프레임: 4204
   길이: 140.3초
🔍 YOLO 분석 진행 중...
진행률: 10%
진행률: 20%
진행률: 30%
진행률: 40%
진행률: 50%
진행률: 60%
진행률: 70%
진행률: 80%
진행률: 90%
진행률: 100%

📊 분석 완료! 결과 통계
총 처리 프레임: 4204
총 탐지된 객체: 8114
객체가 탐지된 프레임: 2808
프레임당 평균 탐지 수: 1.93
객체 탐지율: 66.8%

✅ 결과 영상 저장: result_video_youtube.mp4
🎥 결과 영상 재생:
